# 14 Text Classification

In [1]:
import sys

if "google.colab" in sys.modules:
    print("Running in Colab!")
    !pip install unidecode

In [ ]:
import os
import re
import shutil
import string
import random
import pathlib
import unidecode

import numpy as np
import tensorflow as tf

from IPython.core.magic import register_cell_magic

os.environ["KERAS_BACKEND"] = "jax"

import keras

@register_cell_magic
def backend(line, cell):
    current, required = os.environ.get("KERAS_BACKEND", ""), line.split()[-1]
    if current == required:
        get_ipython().run_cell(cell)
    else:
        print(
            f"This cell requires the {required} backend. To run it, change KERAS_BACKEND to "
            f"\"{required}\" at the top of the notebook, restart the runtime, and rerun the notebook."
        )

---

## Natural-language processing (NLP): The bird's eye view

The **big** story of recent years has been a move away from **hand-crafted features**.

Well until the 1990s the industry employed many linguists to implement specialist knowledge into rules for NLP systems.

Then, statistical approaches took over, especially with the rise of deep learning.

We leave aside the question of exactly **what** current systems understand from language.

What we know is that we can now train them to take in pieces of text, execute task on them.

- "What’s the topic of this text?" (**text classification**)  
- "Does this text contain abuse?" (**content filtering**)  
- "Does this text sound positive or negative?" (**sentiment analysis**)  
- "What should be the next word in this incomplete sentence?" (**language modeling**)  
- "How would you say this in German?" (**translation**)  
- "How would you summarize this article in one paragraph?" (**summarization**)


## Preparing text data

### Method

- **standardisation**: make your text easier to process
- **tokenisation**: split into chunks
- **indexing**: turn chunks into numbers

---

### Text encoding

The question of **how language is represented** in computers is an important topic: if you're interested in working with language, do go and learn this!

The [**The Unicode Standard**](https://en.wikipedia.org/wiki/Unicode) is a world-wide consensus to associate characters of all languages with a unique identifier (a number), called a **code point**.

The Python documentation on [Unicode](https://docs.python.org/3/howto/unicode.html).

**Code points** can then be **encoded** in bits ([**UTF-8**](https://en.wikipedia.org/wiki/UTF-8), 8 bits, is one of the ways you can do that, and is backward-compatible with [**ASCII**](https://en.wikipedia.org/wiki/ASCII), 7 bits).

---

### Text standardization

- Convert to **lower case**;  
- Remove **punctuation**;
- Convert **special characters** to a standard form (é → e);
- *Stemming*: turn **inflected forms** into base ones ("stares"/"stared"/"staring" → "[stare]")

In [ ]:
s = "I didn't wake up this morning. How did that happen?"

In [ ]:
s.lower()

"i didn't wake up this morning. how did that happen?"

In [ ]:
"".join(char for char in s.lower() if char not in string.punctuation) # removing punctuation

'i didnt wake up this morning how did that happen'

In [ ]:
def str_to_ints(s):
    s_enc = s.encode()
    print(f"{s} | {s_enc} | {list(s_enc)}") # encode into bytes, then turn into a list
s = "Jérémie"
str_to_ints(s)
str_to_ints(s.lower())
str_to_ints(unidecode.unidecode(s.lower())) # removing anything

Jérémie | b'J\xc3\xa9r\xc3\xa9mie' | [74, 195, 169, 114, 195, 169, 109, 105, 101]
jérémie | b'j\xc3\xa9r\xc3\xa9mie' | [106, 195, 169, 114, 195, 169, 109, 105, 101]
jeremie | b'jeremie' | [106, 101, 114, 101, 109, 105, 101]


---

### Text splitting (tokenization)

- **word-level tokenization**: split on spaces;
- **n-gram tokenization**: same as above, but making groups of length n;
- **character-level tokenization**: split on characters;
- **byte-level tokenization**: split on bytes.

These (except perhaps the last one) are forms of feature engineering, especially if combined with standardisation.

**Note**: algorithms such as [*byte-pair encoding*](https://huggingface.co/course/chapter6/5?fw=pt) (BPE) or [WordPiece](https://huggingface.co/course/chapter6/6?fw=pt) combine the best of both worlds by processing a corpus at the byte level, and gradually merging frequent co-occurrences, parts of words, all the way up to whole words or even phrases.

In [ ]:
s = "the quick brown fox jumps over the lazy dog"
s_split = s.split(" ")
print(s)
print(s_split)

the quick brown fox jumps over the lazy dog
['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']


In [ ]:
def n_grams(l, n=2):
    n_grams = []
    for i in range(0, len(s_split) - n + 1):
        n_grams.append(s_split[i:i+n])
    return n_grams

In [ ]:
n_grams(s_split, n=2) # each token is two words

[['the', 'quick'],
 ['quick', 'brown'],
 ['brown', 'fox'],
 ['fox', 'jumps'],
 ['jumps', 'over'],
 ['over', 'the'],
 ['the', 'lazy'],
 ['lazy', 'dog']]

In [ ]:
n_grams(s_split, n=3) # each token is three words

[['the', 'quick', 'brown'],
 ['quick', 'brown', 'fox'],
 ['brown', 'fox', 'jumps'],
 ['fox', 'jumps', 'over'],
 ['jumps', 'over', 'the'],
 ['over', 'the', 'lazy'],
 ['the', 'lazy', 'dog']]

---

### Vocabulary indexing

From a dataset, we create a big table **associating each word with a number**.

We can also add special information like an index for:
- **unknown words** ("\[UNK\]" for *unknown*, aka OOV: out of vocabulary);
- **beginning of sentence** ("bos");
- **end of sentence** ("eos");
- **padding** ("\[PAD\]", to make sequences the same length);
- **masked token**, and other customised "special tokens" depending on needs...

In [ ]:
vocab = {token:i for i,token in enumerate(sorted(list(set(s_split))))}
vocab

{'brown': 0,
 'dog': 1,
 'fox': 2,
 'jumps': 3,
 'lazy': 4,
 'over': 5,
 'quick': 6,
 'the': 7}

In [ ]:
print([vocab[word] for word in s_split])

[7, 6, 0, 2, 3, 5, 7, 4, 1]


In [ ]:
print([vocab[word] for word in "the quick fox".split(" ")])

[7, 6, 2]


<!-- <img style="height: 400px" src="images/chollet/figure14.1.png"> -->
<img style="height: 400px" src="https://raw.githubusercontent.com/jchwenger/AI/main/lectures/07/images/chollet/figure14.1.png">

[DLWP](https://deeplearningwithpython.io/chapters/chapter14_text-classification/#preparing-text-data), Figure 14.1

In [ ]:
class Vectorizer:
    """
    An implementation of the transformations in the above schema.
    """
    def standardize(self, text):
        """Make lowercase and remove punctuation"""
        text = text.lower()                                # Python built-in list of punctuation characters
        return "".join(char for char in text if char not in string.punctuation)

    def tokenize(self, text):
        """Standardize then split on space"""
        text = self.standardize(text)
        return text.split()

    def make_vocabulary(self, dataset):
        """Generate token vocabulary from text"""
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            tokens = self.tokenize(text) # this will standardize
            for token in tokens:                                  # first element will be 2
                if token not in self.vocabulary:                  # then len(vocab) is 3 →
                    self.vocabulary[token] = len(self.vocabulary) # next token is 3, etc.
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        """Encode string to tokens"""
        tokens = self.tokenize(text) # this will standardize
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        """Decode tokens to string"""
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

    def one_hot_encode(self, encoded_sequence):
        vector = np.zeros((len(encoded_sequence), len(self.vocabulary)))
        for i, token in enumerate(encoded_sequence):
            vector[i, token] = 1
        return vector

    def one_hot_decode(self, one_hot_sequence):
        return list(np.where(one_hot_sequence == 1)[1])

In [ ]:
vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
vectorizer.make_vocabulary(dataset)

In [ ]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)

[2, 3, 5, 7, 1, 5, 6]


In [ ]:
one_hot_sentence = vectorizer.one_hot_encode(encoded_sentence)
print(one_hot_sentence)

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [ ]:
encoded_sentence = vectorizer.one_hot_decode(one_hot_sentence)
print(encoded_sentence)

[2, 3, 5, 7, 1, 5, 6]


In [ ]:
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


---

### Using the TextVectorization layer

Documentation for [`keras.layers.TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization)

In [ ]:
text_vectorization = keras.layers.TextVectorization(
    output_mode="int" # returns a sequence of integers
)

A lot of options available in the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization).

In [ ]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]                # ↓ the `adapt()` method is the equivalent to the `fit()` one for models
text_vectorization.adapt(dataset)

In [ ]:
vocabulary = text_vectorization.get_vocabulary()
print(vocabulary) # displaying the vocabulary

['', '[UNK]', 'erase', 'write', 'then', 'rewrite', 'poppy', 'i', 'blooms', 'and', 'again', 'a']


In [ ]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [ ]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence) # 'still' was not present in the original corpus (our three sentences)

i write rewrite and [UNK] rewrite again


**Note**

How do you customize such a layer? You can pass user-defined functions to it.

(Documentation for the [`tf.strings`](https://www.tensorflow.org/api_docs/python/tf/strings) module)  
(Regular expressions are *very* useful for any text manipulation: tutorials [here](https://realpython.com/regex-python/), [here](https://realpython.com/regex-python-part-2/) and [here](https://docs.python.org/3/howto/regex.html))

In [ ]:
def custom_standardization_fn(string_tensor):
    """Standardize string tensor using regex (REGular EXpressions)"""
    lowercase_string = tf.strings.lower(string_tensor)               # to lower case
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]", ""   # remove all punctuation
    )

def custom_split_fn(string_tensor):
    """Split string tensor"""
    return tf.strings.split(string_tensor)                           # split on spaces

text_vectorization = keras.layers.TextVectorization(
    output_mode="int",
    standardize=custom_standardization_fn, # apply special functions to
    split=custom_split_fn,                 # standardize and split
)

---

## Two approaches for representing groups of words: Sets and sequences

- Sets: "bag of words", the order is lost.  
- Sequences: order/time matters!

### Preparing the IMDB movie reviews data

#### Downloading the data

In [ ]:
DATASET_DIR = pathlib.Path("aclImdb")

if not DATASET_DIR.exists():
    !curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    !tar -xf aclImdb_v1.tar.gz # this untars the archive to a folder called aclImdb
    !rm -r aclImdb/train/unsup

MODELS_DIR = pathlib.Path("models")
MODELS_DIR.mkdir(exist_ok=True)

In [ ]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

#### Process using `text_dataset_from_directory`

In [ ]:
# code to split the data into train/val folders

TRAIN_DIR = DATASET_DIR / "train"
VAL_DIR = DATASET_DIR / "val"
TEST_DIR = DATASET_DIR / "test"
for category in ("neg", "pos"):
    if not os.path.isdir(VAL_DIR / category):    # do this only once
        os.makedirs(VAL_DIR / category)          # make 'neg'/'pos' dir in validation
        files = os.listdir(TRAIN_DIR / category) # list files in 'train'
        random.Random(1337).shuffle(files)       # shuffle using a seed
        num_val_samples = int(0.2 * len(files))  # 2% of our samples for validation
        val_files = files[-num_val_samples:]
        for fname in val_files:                  # move our files
            shutil.move(TRAIN_DIR / category / fname,
                        VAL_DIR / category / fname)

In [ ]:
BATCH_SIZE = 32

# Keras helper functions creating Tensorflow Dataset
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
# The documentation:
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/text_dataset_from_directory

# each of these iterables returns tuples containing two tensors:
# samples, shape: (batch_size, sample_shape) ← our texts
# targets, shape: (batch_size,)              ← 0 or 1
train_ds = keras.utils.text_dataset_from_directory(
    TRAIN_DIR, batch_size=BATCH_SIZE
)
val_ds = keras.utils.text_dataset_from_directory(
    VAL_DIR, batch_size=BATCH_SIZE
)
test_ds = keras.utils.text_dataset_from_directory(
    TEST_DIR, batch_size=BATCH_SIZE
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
for inputs, targets in train_ds:            # displaying the shapes and dtypes of the first batch
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print()
    print("targets[0]:", targets[0].numpy())
    print()
    print("inputs[0]:", inputs[0].numpy())
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>

targets[0]: 0

inputs[0]: b"This film is awful. Not offensive but extremely predictable. The movie follows the life of a small town family in the mid-60's. The father, the principal at the school, is going through a mid-life crisis. Enter a pretty teacher from the big city who starts challenging her students' minds with some thought-provoking stuff, like think for yourself. The principal doesn't agree with her teaching but she is pretty. You can connect the dots. His teenage daughter (Winona Ryder wannabe Tara Frederick) is fed up with the small town lifestyle and wants to live. She gets some bad advice, hangs out with some bad boys and apparently family planning wasn't being taught at her school. Shocking! Seeing that director Paul Shapiro has mainly worked in TV, this movie plays like a more adult version of an after-school special or a very special episode of one of the more mun

---

## Processing words as a set: The bag-of-words approach

### Single words (unigrams) with binary encoding

- unigram: first you split, then take each split on its own
- binary encoding: multi-hot
- the order is lost

In [ ]:
set("the cat sat on the mat".split()) # unigram == bag of words (BOW)

{'cat', 'mat', 'on', 'sat', 'the'}

In [ ]:
text_vectorization = keras.layers.TextVectorization( # preprocessing our datasets with a `TextVectorization` layer
    max_tokens=20000,                                   # cap the vocab size to 20k
    output_mode="multi_hot",                            # a 1 at the index of each word present, 0 otherwise
)

In [ ]:
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

In [ ]:
vocabulary = text_vectorization.get_vocabulary()
inverse_vocab = dict(enumerate(vocabulary))

In [ ]:
cat = text_vectorization("the cat")
print(cat.numpy(), cat.shape)                                    # "the cat" is now a vector of 20000 zeros
                                                                 #  with 1s at the index of "the" and "cat"
print()
indz = tf.where(cat > 0)                                         # retrieving the indices of the words
print(indz.numpy())
print()
print(" ".join([inverse_vocab[i.numpy().item()] for i in indz])) # retranslating back into a string

[0 1 0 ... 0 0 0] (20000,)

[[   1]
 [1161]]

the cat


In [ ]:
cat = text_vectorization("the cute cat")
print(cat.numpy(), cat.shape)
print()
indz = tf.where(cat > 0)
print(indz.numpy())
print()
print(" ".join([inverse_vocab[i.numpy().item()] for i in indz]))

[0 1 0 ... 0 0 0] (20000,)

[[   1]
 [1020]
 [1161]]

the cute cat


---

### Training a simple model with unigrams

In [ ]:
binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4  # adding a casting to float as the new TF version returns int64
)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4
)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4
)

In [ ]:
for inputs, targets in binary_1gram_train_ds: # inspecting the output of our binary unigram dataset
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print()
    print("targets[0]:", targets[0].numpy())
    print()
    print("inputs[0]:", inputs[0].numpy())
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>

targets[0]: 0

inputs[0]: [1 1 0 ... 0 0 0]


In [ ]:
def get_model(max_tokens=20000, hidden_dim=16, clear=True):
    """
    Our model-building utility
    """
    if clear:
        keras.backend.clear_session()
    inputs = keras.Input(shape=(max_tokens,))
    x = keras.layers.Dense(hidden_dim, activation="relu")(inputs)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy", # our labels are only 0 or 1 (negative/positive)
        metrics=["accuracy"]
    )
    return model

In [ ]:
# Training the binary unigram model
model = get_model()
model.summary()

callbacks = [ # using a callback to save our model
    keras.callbacks.ModelCheckpoint(
        str(MODELS_DIR / "binary_1gram.keras"),
        save_best_only=True
    )
]

model.fit(
    binary_1gram_train_ds.cache(),
    validation_data=binary_1gram_val_ds.cache(),
    epochs=10,
    callbacks=callbacks
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 563s 898ms/step - accuracy: 0.7727 - loss: 0.4855 - val_accuracy: 0.8712 - val_loss: 0.3115
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 7ms/step - accuracy: 0.8972 - loss: 0.2767 - val_accuracy: 0.8812 - val_loss: 0.2989
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9144 - loss: 0.2383 - val_accuracy: 0.8784 - val_loss: 0.3159
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9231 - loss: 0.2250 - val_accuracy: 0.8806 - val_loss: 0.3268
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9286 - loss: 0.2249 - val_accuracy: 0.8798 - val_loss: 0.3427
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9296 - loss: 0.2112 - val_accuracy: 0.8766 - val_loss: 0.3549
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9350 - loss: 0.2054 - val_accuracy: 0.8734 - val_loss: 0.3695
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9389 - loss: 0.1957 - val_accurac

In [ ]:
# And testing it
model = keras.models.load_model(MODELS_DIR / "binary_1gram.keras")
_, acc = model.evaluate(binary_1gram_test_ds)
print(f"Test acc: {acc:.3f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 508s 649ms/step - accuracy: 0.8823 - loss: 0.2938
Test acc: 0.882


In [ ]:
del (
    model, text_vectorization,
    binary_1gram_train_ds, binary_1gram_val_ds, binary_1gram_test_ds
)

----

### Bigrams with binary encoding

- bigram: first you split, then take tokens two by two   
- binary encoding: multi-hot (each bigram is one token!)
- *almost all* order is lost

In [ ]:
text_vectorization = keras.layers.TextVectorization(
    ngrams=2, # ← returns bigrams
    max_tokens=20000, output_mode="multi_hot",
)

In [ ]:
text_vectorization.adapt(["I write, erase, rewrite","Erase again, and then","A poppy blooms.",])
vocabulary = text_vectorization.get_vocabulary()
print(f"{len(vocabulary)} tokens: {vocabulary}") # displaying the vocabulary

19 tokens: ['[UNK]', 'erase', 'write erase', 'write', 'then', 'rewrite', 'poppy blooms', 'poppy', 'i write', 'i', 'erase rewrite', 'erase again', 'blooms', 'and then', 'and', 'again and', 'again', 'a poppy', 'a']


In [ ]:
encoded_sentence = text_vectorization("I write, rewrite, and still rewrite again")
print(encoded_sentence)

tf.Tensor([1 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 1 0 0], shape=(19,), dtype=int64)


---

### Training a simple model with bigrams

In [ ]:
text_vectorization = keras.layers.TextVectorization(
    ngrams=2, # ← returns bigrams
    max_tokens=20000, output_mode="multi_hot",
)

text_vectorization.adapt(text_only_train_ds)

# prepare our datasets
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [ ]:
# Training the binary bigram model
model = get_model()
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint(
        str(MODELS_DIR / "binary_2gram.keras"),
        save_best_only=True
    )
]

model.fit(
    binary_2gram_train_ds.cache(),
    validation_data=binary_2gram_val_ds.cache(),
    epochs=10,
    callbacks=callbacks
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 1016s 2s/step - accuracy: 0.7962 - loss: 0.4498 - val_accuracy: 0.8910 - val_loss: 0.2777
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9164 - loss: 0.2353 - val_accuracy: 0.8916 - val_loss: 0.2858
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9377 - loss: 0.1893 - val_accuracy: 0.8924 - val_loss: 0.3068
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9451 - loss: 0.1714 - val_accuracy: 0.8896 - val_loss: 0.3230
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9492 - loss: 0.1581 - val_accuracy: 0.8890 - val_loss: 0.3471
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9564 - loss: 0.1560 - val_accuracy: 0.8924 - val_loss: 0.3585
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9570 - loss: 0.1473 - val_accuracy: 0.8838 - val_loss: 0.3800
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9585 - loss: 0.1451 - val_accuracy: 

In [ ]:
#  and testing
model = keras.models.load_model(MODELS_DIR / "binary_2gram.keras")
_, acc = model.evaluate(binary_2gram_test_ds)
print(f"Test acc: {acc:.3f}") # better than the unigram model

782/782 ━━━━━━━━━━━━━━━━━━━━ 977s 1s/step - accuracy: 0.8987 - loss: 0.2649
Test acc: 0.899


In [ ]:
del (
    model, text_vectorization,
    binary_2gram_train_ds, binary_2gram_val_ds, binary_2gram_test_ds
)

---

### Save models to Google Drive


In [ ]:
EXPORT=False

if EXPORT:
    # zip models
    !zip aclImdb.models.zip {MODELS_DIR}/*
    # connect to drive
    from google.colab import drive
    drive.mount('/content/drive')
    # copy zip to drive (adjust folder as needed)
    !cp aclImdb.models.zip drive/MyDrive/gold/IS53024B-Artificial-Intelligence/models

## Summary

### NLP

- **standardisation** / **tokenisation** / **indexing**
- tokenisation: various possible **levels** (words, characters, n-grams, etc.)

- **Two kinds of NLP models**:
  - **bag-of-words** sets of words or n-grams, the order is lost.
  - **sequence models** that process word order.